In [1]:
from __future__ import annotations

import random

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
from deeptensor import (
    SGD,
    AdaGrad,
    Adam,
    LinearLayer,
    GeLu,
    LeakyReLu,
    Model,
    Momentum,
    ReLu,
    RMSprop,
    Sigmoid,
    SoftMax,
    Tanh,
    Tensor,
    Value,
    cross_entropy,
    mean_squared_error,
)


In [3]:
np.random.seed(1337)
random.seed(1337)

In [ ]:
# make up a dataset

from sklearn.datasets import make_moons, make_blobs

X, y = make_moons(n_samples=100, noise=0.1)

# y_new = y * 2 - 1  # make y be -1 or 1
# visualize in 2D
plt.figure(figsize=(5, 5))
plt.scatter(X[:, 0], X[:, 1], c=y, s=20, cmap="jet")

In [ ]:
X[0], y[0], X[1], y[1], X[2], y[2]

In [ ]:
# initialize a model
model = Model(
    [
        LinearLayer(2, 16),
        ReLu(),
        LinearLayer(16, 16),
        ReLu(),
        LinearLayer(16, 2),
    ],
    False,  # using_cuda
)

print(model)

In [13]:
optimizer = Adam(model, 0.001)

In [14]:
# loss function
def my_loss_fn():
    scores = []
    total_loss = 0
    accuracy = []
    real_output_ohe = Tensor([2])  # one_hot_encoded
    for i in range(2):
        real_output_ohe.set(i, Value(0))

    for idx, x in enumerate(X):
        inp_tensor = Tensor([2])
        inp_tensor.set(0, Value(x[0]))
        inp_tensor.set(1, Value(x[1]))
        out = model(inp_tensor)
        real_output_ohe.set(y[idx], Value(1))
        # print(f"{out=}; {out.get(0)=}; {out.get(1)=}; {real_output_ohe=}")
        losses = cross_entropy(out, real_output_ohe)
        # print(f"{losses=}")
        real_output_ohe.set(y[idx], Value(0))

        total_loss += losses.data
        # print("-- going for backprop --")
        # Backpropagation
        losses.backward()
        optimizer.step()
        optimizer.zero_grad()
        # # Also get accuracy
        if (out.get(0).data > out.get(1).data and y[idx] == 0) or (
            out.get(0).data < out.get(1).data and y[idx] == 1
        ):
            accuracy.append(1)
        else:
            accuracy.append(0)

    return total_loss, sum(accuracy) / len(accuracy)

In [ ]:
total_loss, acc = my_loss_fn()

acc, total_loss

In [ ]:
# optimization
for k in range(200):
    # forward
    total_loss, acc = my_loss_fn()

    print(f"step {k} loss {total_loss}, accuracy {acc*100}%")

In [ ]:
# visualize decision boundary

h = 0.25
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Xmesh = np.c_[xx.ravel(), yy.ravel()]
inputs = [list(map(Value, xrow)) for xrow in Xmesh]

tensor_input = [Tensor([2]) for _ in range(len(inputs))]
for i in range(len(inputs)):
    tensor_input[i].set(0, inputs[i][0])
    tensor_input[i].set(1, inputs[i][1])

scores = list(map(model, tensor_input))
Z = np.array([s.get(0).data > 0 for s in scores])
Z = Z.reshape(xx.shape)

fig = plt.figure()
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())